In [2]:
!pip install lightning -q

In [3]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import torch
import functools
from transformers import AdamW, get_linear_schedule_with_warmup
import math
from time import time
from datasets import load_dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
path_train = "/kaggle/input/eli5-explain-like-i-am-5/ELI5-001.jsonl"
path_val = "/kaggle/input/eli5-explain-like-i-am-5/ELI5_val.jsonl"
dataset_train = load_dataset('json', data_files = path_train)
dataset_val = load_dataset('json', data_files = path_val)
train = dataset_train['train'].select(range(1,30000))
val = dataset_val['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d431cd6eeda61f75/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-6de20c07f97710b3/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
def replace_text(string):
    return string.replace("\'","").replace("\n","").replace("URL_0","").lower().strip()
def preprocess(ex, n):
    ex['question'] = replace_text(ex['question'])
#     context = [k[0] for k in ex['ctxs'][:3]]
#     context = replace_text(' '.join(context))
    context = ex['ctxs'][:n]
    if type((context[0])) == list:
        context = [k[0] for k in context]
    context = replace_text(' '.join(context))
    ex['ctxs'] = context
    ex['answers'] = [replace_text(i) for i in ex['answers']]
    return ex

In [6]:
train1 = train.map(lambda ex: preprocess(ex,n=3), remove_columns = ['question_id'])
val1 = val.map(lambda ex: preprocess(ex,n=3), remove_columns = ['question_id'])

  0%|          | 0/29999 [00:00<?, ?ex/s]

  0%|          | 0/1507 [00:00<?, ?ex/s]

In [21]:
class eli5dataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.qa_id_list = [
            (i,j)
            for i, qa in enumerate(self.data)
            for j, a in enumerate(qa['answers'])
            if j <= 3
        ]
    def __len__(self):
        return len(self.qa_id_list)
    def make_example(self, idx):
        i,j = self.qa_id_list[idx]
        question = self.data['question'][i]

        context = self.data['ctxs'][i]

        answer = self.data['answers'][i][j]

        return (question, context, answer)
    def __getitem__(self, idx):
        return self.make_example(idx)

In [22]:
def make_qa_s2s_batch(qa_list, tokenizer, max_len=64, max_a_len=360):
    q_ls = (q for q, c, a in qa_list)
    c_ls = (c for q, c, a in qa_list)
    a_ls = (a for q, c, a in qa_list)

    q_toks = tokenizer.batch_encode_plus(q_ls, c_ls, max_length=max_len, padding='max_length', truncation=True, return_tensors='pt')
    q_ids, q_mask = (
        torch.LongTensor(q_toks['input_ids']),
        torch.LongTensor(q_toks['attention_mask'])
    )
    a_toks = tokenizer.batch_encode_plus(a_ls, max_length=min(max_len, max_a_len), padding='max_length', truncation=True,return_tensors='pt')
    a_ids, a_mask = (
        torch.LongTensor(a_toks['input_ids']),
        torch.LongTensor(a_toks['attention_mask'])
    )
    labels = a_ids
    labels[labels == 0] = -100
    
#     print('q_ids shape',q_ids.shape)
#     print('q_mask shape', q_mask.shape)
#     print('a_ids shape', a_ids.shape)
#     print('a_mask shape', a_mask.shape)
#     print("labels shape", labels.shape)
    
    model_inputs = {
        'input_ids': q_ids,
        'attention_mask': q_mask,
        'decoder_attention_mask': a_mask,
        'labels': labels,
    }
    return model_inputs

In [23]:
def data_loader(dataset, args):
    train_sampler = RandomSampler(dataset)
    model_collate_fn = functools.partial(make_qa_s2s_batch, tokenizer=tokenizer, max_len=args.max_length)
    data_loader = DataLoader(dataset, batch_size=args.batch_size, sampler=train_sampler, collate_fn=model_collate_fn)
    return data_loader

In [39]:
class ArgumentsS2S():
    def __init__(self):
        self.batch_size = 8
        self.max_length = 256

s2s_args = ArgumentsS2S()

In [47]:
import lightning as L
class bart_model(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained("t5-base")
    def forward(self, batch_input):
        output = self.model(**batch_input)
        return output.loss, output.logits
    def training_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('train_loss',loss, prog_bar=True, logger=True)
        print('train_loss:', loss)
        return loss
    def validation_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('val_loss',loss, prog_bar=True, logger=True)
        print('val_lss:', loss)
        return loss
    def test_step(self, batch, batch_idx):
        loss, output = self(batch)
        self.log('test_loss',loss, prog_bar=True, logger=True)
        return loss
    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=2e-4)

In [41]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [42]:
train2 = eli5dataset(train1)
val2 = eli5dataset(val1)
train_1 = data_loader(train2, s2s_args)
val_1 = data_loader(val2, s2s_args)

In [44]:
my_model = bart_model()

In [45]:
torch.cuda.empty_cache()
trainer = L.Trainer(accelerator='gpu', devices=2, max_epochs=3)
trainer.fit(my_model, train_1, val_1)

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
INFO: Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
INFO: ----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------

INFO: LOCAL_RANK: 1 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
INFO: 
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Tota

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:480: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:432: PossibleUserWarning: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Exception ignored in: <function _xla_gc_callback at 0x7d8eeb2fdd80>
Exception ignored in: Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
<function _xla_gc_callback at 0x7d8eeb2fdd80>
    Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 103, in _xla_gc_callback
    def _xla_gc_callback(*args):def _xla_gc_callback(*args):
KeyboardInterrupt: 

KeyboardInterrupt: 
